In [41]:
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import fcluster
import time
import pandas as pd
from matplotlib.pyplot import figure
from matplotlib import rc
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from yellowbrick.cluster import SilhouetteVisualizer
from yellowbrick.cluster import KElbowVisualizer
from sklearn import mixture
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN

In [7]:
# Hierarchical clustering:
# defining a function (dendro) to do Hierarchical clustering
def dendro(df, leaf_labels):
# df is input dataset
# leaf_labels is the cells list 
  Df = df.values
  if len(leaf_labels) != len(Df):
    Df = np.transpose(Df)
# linkage with ward metod and euclidean metric:
  Z = linkage(Df, method='ward', metric='euclidean')
#plot the dendrogram
  plt.figure(figsize=(50, 30))
  ax = plt.subplot()
  plt.subplots_adjust(left=0.07, bottom=0.3, right=0.98, top=0.95,wspace=0, hspace=0)
  plt.xlabel('Cell Line')
  plt.ylabel('Distance')
  dendrogram(Z, leaf_rotation=90., leaf_font_size=20., labels=leaf_labels)
  plt.savefig('dendrogram_nci60.png')
# clusters are the output labels
  clusters = fcluster(Z, 60000, criterion='distance')
# fcluster is a function to generate output labels 
  return clusters

In [24]:
#K-means clustering
# input dataset is df
# k is the number of clusters
def kmeansb(df,k):
# PCA to show the clusterings in 2d plot
  g = PCAb(df)
  kmeans = KMeans(n_clusters=k, n_init=400, algorithm="auto") 
# compute K-means clustering
  kmeans1 = kmeans.fit(g) 
# labels are cluster labels for data points
  labels = kmeans1.predict(g)
# cluster centers: 
  C = kmeans1.cluster_centers_ 
  return [g, labels, C, kmeans]

In [15]:
# Pre Processing
# Performing PCA on the data, for dimensionality reduction:
# PCAb is a function to reduce dataset dimentions to 2D
def PCAb(df):
  Df = df.values
  Df = np.transpose(Df)
#PCA number of components=2
  pca = PCA(n_components=2)
  pca.fit(df)
  projected = pca.fit_transform(Df)
  return projected

In [ ]:
# reading inout dataset
datafile = 'TPM_values2.csv'
df = pd.read_csv(datafile, sep=',')
list(df.columns.values)
#df.shape


#Actual labels
# plot the clusters with actual labels
fig1 = plt.figure(figsize=(20,20))
ax1 = fig1.add_subplot(111)
proj1=PCAb(df)
#reading actual labels from its file 
decon_combine_t_C_t = pd.read_csv('decon_combine_t_C_t.csv')
decon_combine_t_C_t["subtype"]=decon_combine_t_C_t["subtype"].map({"MES": 0,"OPC": 1, "AC": 2, "NPC":3})
decon_combine_t_C_t["subtype"]=decon_combine_t_C_t["subtype"].fillna(0)
labeles=decon_combine_t_C_t.subtype.to_list()
#assigning color to each subtype lable
cmap = plt.cm.jet
cmaplist = ["Red","Green","gray" ,"Black"]
cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)
ax1.scatter(proj1[:, 0], proj1[:, 1], c=labeles, s=200, cmap=cmap)
fig1.show()
#adding legend to the plot
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color="Red", lw=4),
                Line2D([0], [0], color="Green", lw=4),
                Line2D([0], [0], color="gray", lw=4),
                Line2D([0], [0], color="Black", lw=4)]
ax1.legend(custom_lines, ['MES', 'OPC', 'AC', 'NPC'], loc=1)


####################################################################################

                        #Clustering Methods
                      
####################################################################################
# Hierarchical clustering:
# Generating dendrogram by calling dendro function
font = {'size'   : 50}
rc('font', **font)
figure(figsize=(200, 160))
cells = list(df.columns.values) 
t1=time.time()
clusters=dendro(df, cells)
t2=time.time()
Hierarchicaltime= t2-t1
#Plotting Hierarchical clustering:
#PCA pre processing:
p= PCAb(df)
#changing the font size in figure
font = {'size'   : 30}
rc('font', **font)
# plot points with cluster dependent colors
plt.scatter(p[:,0], p[:,1], c=clusters, s=200)  
from matplotlib.lines import Line2D
plt.show()



# K-means clustering, k=3
#number of clusters
k=3
t1=time.time()
[proj, labels1, centroids, kmeans] = kmeansb(df,k)
t2=time.time()
#computation time
kmeans3time=t2-t1
fig1 = plt.figure(figsize=(20,20))
ax1 = fig1.add_subplot(111)
# plot the projected data with assigned clusters
ax1.scatter(proj[:, 0], proj[:, 1], c=labels1, s=200, cmap=cmap)
# plot the centroids
ax1.scatter(centroids[:,0], centroids[:,1], c=range(k), s=1000,
marker='*')
fig1.show()



# K-means clustering, k=4
#number of clusters
k=4
t1=time.time()
[proj, labels1, centroids, kmeans] = kmeansb(df,k)
t2=time.time()
#computation time
kmeans4time=t2-t1
fig1 = plt.figure(figsize=(20,20))
ax1 = fig1.add_subplot(111)
# plot the projected data with assigned clusters
ax1.scatter(proj[:, 0], proj[:, 1], c=labels1, s=200, cmap=cmap)
# plot the centroids
ax1.scatter(centroids[:,0], centroids[:,1], c=range(k), s=1000,
marker='*')
fig1.show()


#Elbow plot to find optimal k
fig, ax = plt.subplots()
#k=2,3,4,5,6
visualizer = KElbowVisualizer(kmeans, k=(2,7),ax=ax)
visualizer.fit(proj)
ax.set_xticks(range(2,7))
visualizer.show()
plt.show()


#SilhouetteVisualizer:
fig, ax = plt.subplots(2, 2, figsize=(40,20))
#Silhouette plot for k=2,3,4,5
for i in [2,3,4,5]:
    [proj, labels, centroids, kmeans] = kmeansb(df,i)
    q, mod = divmod(i, 2)
    visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick', ax=ax[q-1][mod],fontsize=200)
    visualizer.fit(proj)


#GMM clustering:
t1=time.time()
lb = np.infty
bic = []
#number of clusters=4
n_components_range = [4]
# Fit Gaussian mixture with EM:
for n_components in n_components_range:
    gmm = mixture.GaussianMixture(n_components=n_components)
    Dpc = PCAb(df)
    gmm.fit(Dpc)
    bic.append(gmm.bic(Dpc))
    if bic[-1] < lb:
       lb = bic[-1]
       best_gmm = gmm
bic = np.array(bic)
clustering = best_gmm
pred_label = clustering.predict(Dpc)
t2=time.time()
gmmtime=t2-t1
# plot the clusters
fig1 = plt.figure(figsize=(20,20))
ax1 = fig1.add_subplot(111)
# plot the projected data with assigned clusters
proj1=PCAb(df)
#labels from gMM:
labeles1=pred_label
ax1.scatter(proj1[:, 0], proj1[:, 1], c=labeles1, s=200, cmap=cmap)
fig1.show()



#Spectral Clustering
Dpc = PCAb(df)
t1=time.time()
clustering = SpectralClustering(n_clusters=4,eigen_solver='arpack',n_init=50,assign_labels='kmeans',degree=5).fit(Dpc)
#clustering labels
pred_label = clustering.labels_
#computation time
t2=time.time()
# plot the clusters
fig1 = plt.figure(figsize=(20,20))
ax1 = fig1.add_subplot(111)
# plot the projected data with assigned clusters
#labels from spectral clustering:
labeles1=pred_label
ax1.scatter(Dpc[:, 0], Dpc[:, 1], c=labeles1, s=200, cmap=cmap)
fig1.show()
spectraaltime=t2-t1



#DBSCAN:
#prepeocessing
Dpc = PCAb(df)
#computation time
t1=time.time()
clustering = DBSCAN(eps=3350, min_samples=2, algorithm='ball_tree', metric='minkowski', leaf_size=90, p=2).fit(Dpc)
#predicted labels
pred_label = clustering.labels_
t2=time.time()
DBSCANtime=t2-t1
# plot the clusters
fig1 = plt.figure(figsize=(20,20))
ax1 = fig1.add_subplot(111)
# plot the projected data with assigned clusters
labeles1=pred_label
ax1.scatter(Dpc[:, 0], Dpc[:, 1], c=labeles1, s=200, cmap=cmap)
fig1.show()



In [ ]:
######################################################################################

                                    #Evaluation methods

######################################################################################

from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import fowlkes_mallows_score
from sklearn.metrics.cluster import normalized_mutual_info_score

pada=pd.read_csv('datadata.csv', header=None)
all_labels=pada.to_numpy()
labeles=all_labels[:,6].T
for i in range(6):
  labels1=all_labels[:,i].T
  print(adjusted_rand_score(labels1, labeles))
  print(jaccard_score(labels1, labeles, average='micro'))
  print(normalized_mutual_info_score(labeles, labels1))
  print(fowlkes_mallows_score(labeles, labels1, sparse=False))

In [ ]:
#Py.WGCNA:

#preprocessing:
pd.read_csv('TPM_values21.csv', header=None).T.to_csv('output1.csv', header=False, index=False)
#using GPU
import tensorflow as tf
tf.test.gpu_device_name()
!pip install PyWGCNA
!pip uninstall matplotlib
!pip install -U matplotlib
#restart runtime

#Setupping up PyWGCNA object
import PyWGCNA
geneExp = 'output1.csv'
pyWGCNA_5xFAD = PyWGCNA.WGCNA(name='5xFAD', 
                              species='mus musculus', 
                              geneExpPath=geneExp, 
                              save=True)
pyWGCNA_5xFAD.geneExpr.to_df().head(5)

#Pre-processing workflow:
font = {'size'   : 20}
rc('font', **font)
figure(figsize=(200, 160))
pyWGCNA_5xFAD.preprocess()

#Construction of the gene network and identification of modules
pyWGCNA_5xFAD.findModules()

#using deconvolution final labels to produce modules
pyWGCNA_5xFAD.updateSampleInfo(path='deconv.csv')
# add color for metadata
pyWGCNA_5xFAD.setMetadataColor('subtype', {'MES': 'green', 'AC': 'yellow', 'NPC': 'red', 'OPC': 'blue'})
font = {'size'   : 20}
rc('font', **font)
figure(figsize=(200, 160))
pyWGCNA_5xFAD.analyseWGCNA(order=None, geneList=None, show=True)

#saving modules
pyWGCNA_5xFAD.saveWGCNA()

#reading modules saved file:
import PyWGCNA
pyWGCNA_5xFAD = PyWGCNA.readWGCNA("5xFAD.p")
pyWGCNA_5xFAD.datExpr.var.head(5)

#showing all networks
modules = pyWGCNA_5xFAD.datExpr.var.moduleColors.unique().tolist()
pyWGCNA_5xFAD.CoexpressionModulePlot(modules=modules, numGenes=2000, numConnections=1000, minTOM=0, file_name="all")

#showing one of the networks:
pyWGCNA_5xFAD.CoexpressionModulePlot(modules=["white"])